In [2]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [4]:
connection = sqlite3.connect("../db.sqlite3")

query = """
SELECT *
FROM restaurant_order AS o
JOIN restaurant_orderitem AS oi ON o.id = oi.order_id
JOIN restaurant_product AS p ON oi.product_id = p.id
"""

df = pd.read_sql(query, connection)
connection.close()

df = df.drop(columns=["id", "id", "id"])
df.head(10)

,datetime,quantity,order_id,product_id,price,name
0,2019-08-03 20:25:00,2,1,6,0.80,Plain Papadum
1,2019-08-03 20:25:00,1,1,240,12.95,King Prawn Balti
2,2019-08-03 20:25:00,1,1,19,2.95,Garlic Naan
3,2019-08-03 20:25:00,1,1,36,3.95,Mushroom Rice
4,2019-08-03 20:25:00,1,1,111,8.95,Paneer Tikka Masala
5,2019-08-03 20:25:00,1,1,3,0.50,Mango Chutney
6,2019-08-03 20:17:00,1,2,11,2.60,Plain Naan
7,2019-08-03 20:17:00,1,2,36,3.95,Mushroom Rice
8,2019-08-03 20:17:00,1,2,55,4.95,Tandoori Chicken (1/4)
9,2019-08-03 20:17:00,1,2,98,7.95,Vindaloo - Lamb


# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
dishes = df.nlargest(10, "quantity")
dishes["percent"] = round(dishes["quantity"] / dishes["quantity"].sum() * 100, 1)
fig, ax = plt.subplots()
ax.pie(dishes["percent"], labels=dishes["name"], autopct='%1.1f%%')
ax.set_title("Top 10 position in menu by quantity")

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
items = df[["quantity", "price", "name"]].copy()
items["total_price"] = df["quantity"] * df["price"]
items = items.nlargest(10, "total_price")
fig, ax = plt.subplots()
ax.pie(items["total_price"], labels=items["name"], autopct="%1.1f%%")
ax.set_title("Top 10 products")

# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
order_hour = df[["datetime", "order_id", "price"]].copy()
order_hour["datetime"] = pd.to_datetime(order_hour["datetime"])
order_hour["order_time"] = order_hour["datetime"].dt.hour

hour_income = order_hour.groupby("order_time")["price"].sum()

plt.figure(figsize=(10, 6))
hour_income.plot(kind='bar', color='skyblue')
plt.title("Profit by order hour")
plt.xlabel("Order hour")
plt.ylabel("Total sum")
plt.xticks(rotation=0)
plt.show()


# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
day_orders = df[["datetime", "order_id", "price"]].copy()
day_orders["datetime"] = pd.to_datetime(day_orders["datetime"])
day_orders["week_day"] = day_orders["datetime"].dt.day_name()

day_income = day_orders.groupby("week_day")["price"].sum()

days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
day_income = day_income.reindex(days_order)

plt.figure(figsize=(10, 6))
day_income.plot(kind='bar', color='skyblue')
plt.title("Profit by week day")
plt.xlabel("Days")
plt.ylabel("Total sum")
plt.xticks(rotation=45)
plt.show()